In [1]:
import re
import scrapy
from scrapy.crawler import CrawlerProcess
import logging
import pandas as pd
import numpy as np

:0: UserWarning: You do not have a working installation of the service_identity module: 'No module named 'service_identity''.  Please install it from <https://pypi.python.org/pypi/service_identity> and make sure all of its dependencies are satisfied.  Without the service_identity module, Twisted can perform only rudimentary TLS client hostname verification.  Many valid certificate/hostname mappings may be rejected.


In [2]:
#change date format
def two_digits(x):
    if x<10:
        return str(0) + str(x)
    else:
        return str(x)

In [ ]:
#scrape from sportsbookreview
url_template = 'https://www.sportsbookreview.com/betting-odds/nba-basketball/?date={year}{month}{day}'
start_urls = []
years = list(range(2007, 2020))
months = ['01', '02', '03', '04', '05', '06', '10', '11', '12']
days = list(range(1,32))
for year in years:
    year = str(year)
    for month in months:
        if(year + month > '200709'):
            for day in days:
                day = two_digits(day)
                start_urls.append(url_template.format(year=year, month=month, day=day))

In [ ]:
#use scrapy to find betting data
class OddsScraper(scrapy.Spider):
    name = "BettingOdds"
    start_urls = start_urls
    custom_settings = {
        #'LOG_LEVEL': logging.WARNING,
        'FEED_FORMAT':'json',
        'FEED_URI': 'games_odds.json'
    }
    
    
    #function to find all betting odds data for a given date
    def parse(self, response):
        n = 0
        day = response.url[-2:]
        month = response.url[-4:-2]
        year = response.url[-8:-4]
        odds_dict = {'years' : [], 'months' : [], 'days' : [], 'teams' : [], 'odds' : []}
        for item in response.xpath("//*[@id='bettingOddsGridContainer']/div[contains(@class, '_1eZfC')]\
        /div[contains(@class, '_3A-gC _2DWLf _3zKaX')]//span[contains(@class, '_3O1Gx')]"):
            odds_dict['teams'].append(re.search('>.+<', item.extract())[0][1:-1])
        for item in response.xpath("//*[@id='bettingOddsGridContainer']/div[contains(@class, '_1eZfC')]\
        /div[contains(@class, '_3A-gC _2DWLf _3zKaX')]/div[contains(@class, '_2uJuW')]\
        /div[contains(@class, '_1Nn7e _2-YDr _2ZO4X')]/section[contains(@class, '_2MuQm _1aLEi _2NFWr')]\
        //span[contains(@class, _1QEDd)]"):
            if '_1QEDd' in item.extract():
                odd = ' '
                x = re.findall('>[^/"!]+<', item.extract())
                for char in x:
                    char = char.replace('½', '.5')
                    odd += char[1:-1]
                odds_dict['odds'].append(odd)
                odds_dict['years'].append(year)
                odds_dict['months'].append(month)
                odds_dict['days'].append(day)
        yield odds_dict
process = CrawlerProcess()
process.crawl(OddsScraper)
process.start()

2019-04-04 13:53:01 [scrapy.utils.log] INFO: Scrapy 1.6.0 started (bot: scrapybot)
2019-04-04 13:53:01 [scrapy.utils.log] INFO: Versions: lxml 3.8.0.0, libxml2 2.9.9, cssselect 1.0.1, parsel 1.5.1, w3lib 1.17.0, Twisted 17.5.0, Python 3.6.8 |Anaconda custom (x86_64)| (default, Dec 29 2018, 19:04:46) - [GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)], pyOpenSSL 19.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.6.1, Platform Darwin-17.6.0-x86_64-i386-64bit
2019-04-04 13:53:01 [scrapy.crawler] INFO: Overridden settings: {'FEED_FORMAT': 'json', 'FEED_URI': 'xxxxgames_odds.json'}
2019-04-04 13:53:01 [scrapy.extensions.telnet] INFO: Telnet Password: 5bbdd2fc5e1f3ccc
2019-04-04 13:53:01 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2019-04-04 13:53:01 [scrapy.middlewar

In [ ]:
#read odds data
x = pd.read_json('games_odds.json') # get odds data
x = x[x['days'].map(lambda d: len(d)) > 0] #removes blank lists rows from json
x = x.values.tolist() #convert to list of list from df
y = np.concatenate(x,1) #convert to 2D np array 
odds_df = pd.DataFrame(np.transpose(y), columns=['Day', 'Month', 'Odds', 'Team', 'Year'])
odds_df = odds_df[['Day', 'Month', 'Year', 'Team', 'Odds']].sort_values(['Year', 'Month', 'Day'])

In [ ]:
home_df = pd.read_csv('homedf.csv')

In [ ]:
x = home_df[['Day', 'Month', 'Year']]

In [ ]:
x[x['Month']==10]

In [ ]:
odds_df[odds_df['Month']=='10']

In [ ]:
odds_df.to_csv('oddsdf.csv')